# Evidence Assessment

In [1]:
import pandas as pd
import os
import json
import logging


from utils.General.quicker import Quicker, QuickerData, QuickerStage
from utils.Evidence_Assessment.paper import Paper
from utils.Evidence_Assessment.outcome import Outcome
from utils.Evidence_Assessment.evidence import Evidence
from utils.logging import setup_logging

# setup_logging()

# Hyperparameters
YOUR_CONFIG_PATH = 'config/config.json' #  your config.json file
YOUR_QUESTION_DECOMPOSITION_PATH =  'data/2021ACR RA/Question_Decomposition' # your question decomposition folder
YOUR_LITERATURE_SEARCH_PATH = 'data/2021ACR RA/Literature_Search' # your literature search folder
YOUR_STUDY_SELECTION_PATH = 'data/2021ACR RA/Study_Selection' # your study selection folder
YOUR_EVIDENCE_ASSESSMENT_PATH = 'data/2021ACR RA/Evidence_Assessment' # your evidence assessment folder
YOUR_PAPER_LIBRARY_PATH = 'data/2021ACR RA/Paper_Library' # your paper library folder

pico_idx = "ef0e4f95" # PICO index of the clinical question, saved in your PICO_Information.json file

In [2]:
import shutil

# move the files from the source directory to the target directory
def transfer_outcome_and_paperinfo(source_dir_path, target_dir_path, only_index: str = None):
    '''
    Transfer the 'outcomeinfo' and 'paperinfo' folders and their subfolders/files from source_dir_path to target_dir_path.
    If only_index is provided, only transfer subfolders/files containing '_PICO{only_index}' in their names.

    :param source_dir_path: Source directory path
    :param target_dir_path: Target directory path
    :param only_index: Only transfer subfolders/files containing '_PICO{only_index}' in their names
    '''

    # Transfer 'outcomeinfo' folder
    source_outcomeinfo_path = os.path.join(source_dir_path, 'outcomeinfo')
    target_outcomeinfo_path = os.path.join(target_dir_path, 'outcomeinfo')
    if os.path.exists(source_outcomeinfo_path):
        if only_index:
            for item in os.listdir(source_outcomeinfo_path):
                if f'_PICO{only_index}' in item:
                    source_item_path = os.path.join(source_outcomeinfo_path, item)
                    target_item_path = os.path.join(target_outcomeinfo_path, item)
                    if os.path.isdir(source_item_path):
                        shutil.copytree(source_item_path, target_item_path, dirs_exist_ok=True)
                    else:
                        os.makedirs(target_outcomeinfo_path, exist_ok=True)
                        shutil.copy2(source_item_path, target_item_path)
        else:
            shutil.copytree(source_outcomeinfo_path, target_outcomeinfo_path, dirs_exist_ok=True)

    # Transfer 'paperinfo' folder
    source_paperinfo_path = os.path.join(source_dir_path, 'paperinfo')
    target_paperinfo_path = os.path.join(target_dir_path, 'paperinfo')
    if os.path.exists(source_paperinfo_path):
        if only_index:
                    for item in os.listdir(source_paperinfo_path):
                        if f'_PICO{only_index}' in item:
                            source_item_path = os.path.join(source_paperinfo_path, item)
                            target_item_path = os.path.join(target_paperinfo_path, item)
                            if os.path.isdir(source_item_path):
                                shutil.copytree(source_item_path, target_item_path, dirs_exist_ok=True)
                            else:
                                os.makedirs(target_paperinfo_path, exist_ok=True)
                                shutil.copy2(source_item_path, target_item_path)
        else:
            shutil.copytree(source_paperinfo_path, target_paperinfo_path, dirs_exist_ok=True)



In [3]:
# The confirmed documents need to be transferred to the evidence assessment path for subsequent operations. Here is an example code to transfer the files.
# transfer_outcome_and_paperinfo(source_dir_path=YOUR_STUDY_SELECTION_PATH,target_dir_path=YOUR_EVIDENCE_ASSESSMENT_PATH,only_index=pico_index)

In [4]:
quicker_data = QuickerData(disease='Rheumatoid Arthritis (RA)', pico_idx=pico_idx)
quicker = Quicker(
    config_path=YOUR_CONFIG_PATH,
    question_deconstruction_database_path=YOUR_QUESTION_DECOMPOSITION_PATH,
    literature_search_database_path=YOUR_LITERATURE_SEARCH_PATH,
    study_selection_database_path=YOUR_STUDY_SELECTION_PATH,
    evidence_assessment_database_path=YOUR_EVIDENCE_ASSESSMENT_PATH,
    quicker_data=quicker_data,
    paper_library_base=YOUR_PAPER_LIBRARY_PATH,
)

quicker.comparator_postfix_map = {}

question_deconstruction_datapath = os.path.join(
    quicker.question_deconstruction_database_path, 'PICO_Information.json'
)

question_deconstruction_data = pd.read_json(
    question_deconstruction_datapath, dtype={'Index': str}
)

question_deconstruction_data = question_deconstruction_data[
    question_deconstruction_data['Index'] == pico_idx
]

# print(question_deconstruction_data)
original_qd_dict = question_deconstruction_data.to_dict(orient='records')

data_dict = dict(
    pico_idx=original_qd_dict[0]['Index'],
    # pic=original_qd_dict[0]['Answer'],
    clinical_question=original_qd_dict[0]['Question'],
    population=original_qd_dict[0]['P'],
    intervention=original_qd_dict[0]['I'],
    comparison=original_qd_dict[0]['C'],
    # study=original_qd_dict[0]['S'],
    valid_comparison_list=original_qd_dict[0]['C'],
    outcome=original_qd_dict[0]['O'],
    # valid_comparison_list=original_qd_dict[0]['C'],
    # annotation=annotation,
)


quicker._add_data_to_quickerdata_for_test(
    stage=QuickerStage.STUDY_SELECTION,
    default_value=data_dict,
)


In [5]:

quicker.load_outcome_list()
quicker.load_paper_list()
print(quicker.quicker_data.pico_idx)
print(quicker)
# print(quicker.quicker_data)

print(quicker.quicker_data.check_stage_state())
print(quicker.quicker_data.not_none_data)

ef0e4f95
Quicker: EVIDENCE_ASSESSMENT
StageState.NOT_STARTED
{'disease': 'Rheumatoid Arthritis (RA)', 'clinical_question': 'Should patients with RA on DMARDs who are in low disease activity gradually taper off DMARDs, abruptly withdraw DMARDs, or continue DMARDS at the same doses?', 'pico_idx': 'ef0e4f95', 'population': 'Patients with rheumatoid arthritis (RA) who are on Disease-Modifying Anti-Rheumatic Drugs (DMARDs) and in low disease activity', 'intervention': 'Gradually taper off DMARDs', 'comparison': ['Abruptly withdraw DMARDs or continue DMARDs at the same doses'], 'study': [], 'outcome': {'Abruptly withdraw DMARDs or continue DMARDs at the same doses': ['Disease activity (follow up: range 9 months to 12 months; assessed with: DAS28-ESR)', 'Flare (follow up: range 6 months to 12 months)', 'Radiographic progression (follow up: range 9 months to 12 months; assessed with: mTSS)', 'Fatigue (follow up: range 9 months to 12 months; assessed with: FACIT-F)', 'Quality of Life (follow up

In [6]:
logging.info("Run evidence assessment")
for comparator in getattr(
    quicker.quicker_data, 'valid_comparison_list'
):  # list
    assessed_outcome_list = quicker.assess_evidence(comparator=comparator)